In [1]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque

offene Probleme / mögliche Verbesserungen:
    - in einem Endzustand nicht mehr q(a',s') berechnen
    - Minibatch mit TF erzeugen
    - store old network weights and use them in loss function

In [23]:
# build graph
obs_dim = 4
n_acts = 2
gamma = 0.9

obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None, 1), dtype=tf.int32)
rew_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32)
new_obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)

# make core of state-action-value function network
mlp_action_val = tf.keras.models.Sequential()
mlp_action_val.add(tf.keras.layers.Dense(50, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(50, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(n_acts))

# define state action values
old_state_action_values = mlp_action_val(obs_ph)
new_state_action_values = mlp_action_val(new_obs_ph)

# select action
greedy_action = tf.math.argmax(old_state_action_values)

# define loss function
y = rew_ph + gamma * np.max(new_state_action_values)
action_masks = tf.one_hot(act_ph, n_acts)
old_selected_action_values = tf.reduce_sum(action_masks * old_state_action_values, axis=1)
action_value_loss = tf.reduce_mean((y - old_selected_action_values)**2)

# define optimizer
optimizer_action_value = tf.train.AdamOptimizer(0.0001)
train_action_value = optimizer_action_value.minimize(action_value_loss)


In [28]:
def dqn(sess, env, experience_replay_buffer, n_episodes):
    for i in range(n_episodes):
        done = False
        state = env.reset()
        cnt = 0
        while not done:
            action = select_action(sess, state)
            new_state, reward, done, _ = env.step(action)
            experience_replay_buffer.append((state, action, reward, new_state))
            state = new_state
            update_q_function(experience_replay_buffer)
            cnt += 1
        print(i, cnt)
        
        
def update_q_function(experience_replay_buffer):
    experience_batch = sample_experience_batch(experience_replay_buffer)
    states, actions, rewards, new_states = extract_data(experience_batch)
    loss = gradient_step(states, actions, rewards, new_states)

def gradient_step(states, actions, rewards, new_states):
    _, loss = sess.run([train_action_value, action_value_loss], feed_dict={
                                obs_ph: np.array(states).reshape(-1, obs_dim),
                                act_ph: np.array(actions).reshape(-1,1),
                                rew_ph: np.array(rewards).reshape(-1,1),
                                new_obs_ph: np.array(new_states).reshape(-1,obs_dim)
                             })
    return loss
    
def extract_data(experience_batch):
    states = [x[0] for x in experience_batch]
    actions = [x[1] for x in experience_batch]
    rewards = [x[2] for x in experience_batch]
    new_states = [x[3] for x in experience_batch]
    return states, actions, rewards, new_states
    
def sample_experience_batch(experience_replay_buffer, batch_size=100):
    indices = np.random.choice(np.arange(len(experience_replay_buffer)), batch_size)
    tmp = np.array(experience_replay_buffer)
    sample_batch = tmp[indices, :]
#     sample_batch = np.random.choice(experience_replay_buffer,batch_size, replace = False)
    return sample_batch

def select_action(sess, state):
    action = 0
    if np.random.rand() < 0.1:
        action = np.random.randint(2)
    else:
        action = sess.run(greedy_action, {obs_ph: state.reshape(1,-1)})[0]
    return action

In [30]:
# main
exp_replay_size = 500
experience_replay_buffer = deque([], exp_replay_size)
env = gym.make('CartPole-v0')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

dqn(sess, env, experience_replay_buffer, 1000)

0 9
1 9
2 8
3 9
4 10
5 11
6 10
7 10
8 9
9 9
10 9
11 9
12 14
13 11
14 9
15 10
16 11
17 10
18 9
19 9
20 11
21 9
22 10
23 8
24 10
25 11
26 9
27 12
28 8
29 12
30 10
31 10
32 9
33 11
34 11
35 9
36 10
37 9
38 11
39 10
40 10
41 9
42 9
43 9
44 10
45 10
46 9
47 11
48 9
49 15
50 8
51 8
52 10
53 9
54 10
55 9
56 8
57 9
58 12
59 10
60 10
61 10
62 12
63 11
64 11
65 9
66 11
67 10
68 10
69 9
70 10
71 9
72 9
73 12
74 11
75 9
76 10
77 10
78 8
79 9
80 10
81 11
82 10
83 10
84 10
85 10
86 10
87 9
88 10
89 10
90 10
91 11
92 9
93 8
94 9
95 9
96 11
97 8
98 9
99 10
100 10
101 9
102 10
103 9
104 10
105 13
106 10
107 9
108 11
109 9
110 10
111 11
112 9
113 9
114 10
115 9
116 10
117 9
118 10
119 8
120 11
121 9
122 9
123 9
124 8
125 10
126 9
127 10
128 11
129 9
130 10
131 10
132 9
133 10
134 10
135 11
136 10
137 10
138 8
139 8
140 10
141 10
142 11
143 10
144 12
145 9
146 11
147 9
148 9
149 9
150 9
151 10
152 10
153 8
154 10
155 9
156 10
157 9
158 12
159 9
160 8
161 9
162 11
163 8
164 10
165 9
166 9
167 10
168 10
16